<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher_New/blob/main/cropping_articular_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#アノテーション情報に従って、個々の関節の画像を切り出す。
今回、"xpFilesRightHand"から右手のxpを記録したpngファイルを読み取り、RectLabelを使用して作成したアノテーション情報を用いて、個々の関節の画像を切り出す。
画像ファイルとアノテーション情報を記したjsonファイルが収められたディレクトリは、content/drive/MyDrive/xpFilesRightHandに収まられている。
画像のフォーマットはすべてpngである。ファイル名の0番目から5番目までの文字が被検者のid番号、42番目から47番目までの数字が画像の撮影日となっている。この部分を使って新たなファイル名とディレクトリ名を作成する。
アノテーション情報を記したjsonファイルの名前は"coco.json"とする。アノテーションクラスは、'wrist', 'MCP1st', 'IP', 'PIPs', 'MCPs', 'DIPs'の6クラスである。
さて、"coco.json"を読み取って、画像から、bounding_boxの境界線に沿って画像の一部を切り出す。
切り出した画像は、クラスごとに別のサブディレクトリに格納。6けたのidと6けたの撮影日の間を_で結んだ合計13文字の文字列のあとに、さらに_を置いて、クラス名を添付した文字列をこのサブディレクトリの名称とする。
これらのサブディレクトリを格納するディレクトリの名前は、6けたのidと6けたの撮影日の間を_で結んだ合計13文字の文字列とする。
さらにこのディレクトリすべてを格納するディレクトリの名称は、'articular_cropping'とする。

In [1]:
from google.colab import drive
import os
import shutil
import cv2
import numpy as np
import json
from PIL import Image

In [2]:
# Google Driveを呼び出す。
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


jsonファイルの読み込み

In [9]:
# COCO形式のJSONファイルのパス
json_file_path = '/content/drive/MyDrive/coco.json'

# JSONファイルを読み込む関数
def load_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# JSONファイルからアノテーション情報を読み込む
coco_data = load_json(json_file_path)

画像の切り出し

In [13]:
def crop_and_save_images(coco_data, images_dir, output_dir):
    for img_info in coco_data['images']:
        # 画像ファイル名とパスを取得
        file_name = img_info['file_name']
        img_path = os.path.join(images_dir, file_name)

        # 被検者IDと撮影日を抽出
        subject_id = file_name[:6]
        capture_date = file_name[42:48]
        dir_name = f"{subject_id}_{capture_date}"

        # 画像を読み込む
        img = Image.open(img_path)

        # この画像に関連するアノテーションを見つける
        for ann in coco_data['annotations']:
            if ann['image_id'] == img_info['id']:
                # Bounding Boxを取得して画像を切り出す
                bbox = ann['bbox']
                x, y, width, height = bbox
                cropped_img = img.crop((x, y, x + width, y + height))

                # クラス名とアノテーションIDを取得
                category_id = ann['category_id']
                annotation_id = ann['id']
                category_name = [c['name'] for c in coco_data['categories'] if c['id'] == category_id][0]

                # 保存先ディレクトリを作成
                class_dir = os.path.join(output_dir, dir_name, f"{dir_name}_{category_name}")
                os.makedirs(class_dir, exist_ok=True)

                # 切り出した画像を保存（アノテーションIDをファイル名に追加）
                cropped_file_name = f"{subject_id}_{capture_date}_{category_name}_{annotation_id}.png"
                cropped_img.save(os.path.join(class_dir, cropped_file_name))

# 画像ディレクトリと出力ディレクトリのパス
images_dir = '/content/drive/MyDrive/xpFilesRightHand'
output_dir = '/content/drive/MyDrive/articular_cropping'

crop_and_save_images(coco_data, images_dir, output_dir)

#アノテーションクラス毎に画像を格納する。
以降、異常所見を検出するモデルを作成するために、アノテーションクラス毎にDETRを訓練する。そのためには画像をアノテーションクラス毎にまとめてディレクトリに格納しておいた方が能率がよい。そのような目的に沿って、関数"crop_and_save_images_by_class"を定義し"cropped_joints_for_annotation"というディレクトリに格納した。

In [14]:
def crop_and_save_images_by_class(coco_data, images_dir, output_dir):
    for img_info in coco_data['images']:
        # 画像ファイル名とパスを取得
        file_name = img_info['file_name']
        img_path = os.path.join(images_dir, file_name)

        # 被検者IDと撮影日を抽出
        subject_id = file_name[:6]
        capture_date = file_name[42:48]

        # 画像を読み込む
        img = Image.open(img_path)

        # この画像に関連するアノテーションを見つける
        for ann in coco_data['annotations']:
            if ann['image_id'] == img_info['id']:
                # Bounding Boxを取得して画像を切り出す
                bbox = ann['bbox']
                x, y, width, height = bbox
                cropped_img = img.crop((x, y, x + width, y + height))

                # クラス名とアノテーションIDを取得
                category_id = ann['category_id']
                annotation_id = ann['id']
                category_name = [c['name'] for c in coco_data['categories'] if c['id'] == category_id][0]

                # 保存先ディレクトリを作成
                class_dir = os.path.join(output_dir, category_name)
                os.makedirs(class_dir, exist_ok=True)

                # 切り出した画像を保存（アノテーションIDをファイル名に追加）
                cropped_file_name = f"{subject_id}_{capture_date}_{annotation_id}.png"
                cropped_img.save(os.path.join(class_dir, cropped_file_name))

# 画像ディレクトリと出力ディレクトリのパス
images_dir = '/content/drive/MyDrive/xpFilesRightHand'
output_dir = '/content/drive/MyDrive/cropped_joints_for_annotation'

crop_and_save_images_by_class(coco_data, images_dir, output_dir)


#左手鏡像画像の切り出し
より多くの画像を病変のアノテーションに使用できるようにするため、左手画像の鏡像も同様に切り出して使用する。

In [3]:
# COCO形式のJSONファイルのパス
json_file_path = '/content/drive/MyDrive/mirror_image/coco.json'

# JSONファイルを読み込む関数
def load_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# JSONファイルからアノテーション情報を読み込む
coco_data = load_json(json_file_path)

In [4]:
def crop_and_save_images_by_class(coco_data, images_dir, output_dir):
    for img_info in coco_data['images']:
        # 画像ファイル名とパスを取得
        file_name = img_info['file_name']
        img_path = os.path.join(images_dir, file_name)

        # 被検者IDと撮影日を抽出
        subject_id = file_name[9:15]
        capture_date = file_name[51:57]

        # 画像を読み込む
        img = Image.open(img_path)

        # この画像に関連するアノテーションを見つける
        for ann in coco_data['annotations']:
            if ann['image_id'] == img_info['id']:
                # Bounding Boxを取得して画像を切り出す
                bbox = ann['bbox']
                x, y, width, height = bbox
                cropped_img = img.crop((x, y, x + width, y + height))

                # クラス名とアノテーションIDを取得
                category_id = ann['category_id']
                annotation_id = ann['id']
                category_name = [c['name'] for c in coco_data['categories'] if c['id'] == category_id][0]

                # 保存先ディレクトリを作成
                class_dir = os.path.join(output_dir, category_name)
                os.makedirs(class_dir, exist_ok=True)

                # 切り出した画像を保存（アノテーションIDをファイル名に追加）
                cropped_file_name = f"{subject_id}_{capture_date}_{annotation_id}.png"
                cropped_img.save(os.path.join(class_dir, cropped_file_name))

# 画像ディレクトリと出力ディレクトリのパス
images_dir = '/content/drive/MyDrive/mirror_image'
output_dir = '/content/drive/MyDrive/cropped_joints_for_annotation_mirror_image'

crop_and_save_images_by_class(coco_data, images_dir, output_dir)